## Data 3550 Final Project
### Ayman Boules, Katherine Simon, Nicholas Sartino, Sammi Hamdan

# 1. Import the required packages

In [ ]:
# Installs (comment out if you don't need)

# !pip-install missingno
%pip install seaborn

In [ ]:
# Imports

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,mean_absolute_percentage_error

from numpy import arange

from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

import statsmodels.api as sm

pd.set_option('display.max_columns',200) #allows for up to 200 columns to be displayed when viewing a dataframe
pd.set_option('display.max_rows',100)
# The following was commented out as it is deprecated and no longer works with current version.
#plt.style.use('seaborn') # a style that can be used for plots - see style reference above

# trick to widen the screen
from IPython.core.display import display, HTML

#Widens the code landscape 
display(HTML("<style>.container { width:95% !important; }</style>"))

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Step 1

## Load in the Dataset

In [ ]:
df = pd.read_csv('College_Admission_data.csv')

## Data Exploratory Analysis

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
ax = sns.boxplot(x='Carnegie Classification 2010: Basic',y='Enrolled total', data=df)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.show()

In [ ]:
ax = sns.barplot(x='Carnegie Classification 2010: Basic', y='Applicants total', data=df)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.show()

## Data Preprocessing

In [ ]:
# Splitting the target variable and features
X = df.drop('Carnegie Classification 2010: Basic', axis=1)
y = df['Carnegie Classification 2010: Basic']

In [ ]:
# Convert the categorical target variable into a numerical variable.
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Convert the encoded labels back to the original categories
# y = le.inverse_transform(y_encoded)

# Convert categorical features into numerical features.
df_dummies = pd.get_dummies(X, drop_first=True)
df_dummies.info()

In [ ]:
df_dummies.head()

In [ ]:
# Find columns with missing values and their counts
missing_counts = df_dummies.isnull().sum()

# Filter and print columns with missing values
columns_with_missing_values = missing_counts[missing_counts > 0]
print(columns_with_missing_values)

### Data Imputation
* There are a lot of variables with missing values. Many of them are correlated as well. For example, all of the enrollment variables have exactly 2 missing values.
* The goal in this section will be to try to maintain the representation of the data, which means that no statistical value will be used to fill in an NA.

#### Rules that will be followed:
* columns having at least half of its values being missing values will be dropped.
* columns having a significant number of missing values (>=100) will have its values substituted with the median (assuming a normal distribution).
* columns columns having an insignificant number of missing values will be substituted with appropriate values that indicate that those values were not given.

In [ ]:
# Removing columns with a very large amount of missing values

# Threshold
prop = 0.5

# Drop columns with more than 'prop' proportion of missing values
df_filtered = df_dummies.dropna(thresh=int(df_dummies.shape[0] * prop), axis=1)

In [ ]:
# Handling columns having a significant number of missing values

# Threshold
min_missing_values = 100

# Go through each column in the DataFrame
for col in df_filtered.columns:
    # If the column has 100 or more missing values
    if df_filtered[col].isna().sum() >= min_missing_values:
        # Fill the missing values with the median of the column
        df_filtered[col].fillna(df_filtered[col].median(), inplace=True)

In [ ]:
# Handling columns having an insignificant number of missing values.

# Fill in the rest of the missing values with -1.
df_filtered.fillna(-1, inplace=True)

In [ ]:
# Find columns with missing values and their counts
missing_counts = df_filtered.isnull().sum()

# Filter and print columns with missing values
columns_with_missing_values = missing_counts[missing_counts > 0]
print(columns_with_missing_values)

### Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y_encoded,test_size=0.3,random_state=21)

### Summary of Section
* Categorical variables were converted into numerical variables.
* Missing values were handled according to the rules determined.
* The data was split into training and testing sets.

# Step 2

# Step 3

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Drop 'Name' column from X_train and X_test
X_train = X_train.drop('Name', axis=1)
X_test = X_test.drop('Name', axis=1)

print(X_train.dtypes)
print(X_test.dtypes)

# Identify categorical columns
categorical_columns = X_train.select_dtypes(include=['object']).columns.tolist()

# If categorical columns are present, encode them using pd.get_dummies()
if categorical_columns:
    X_train_encoded = pd.get_dummies(X_train, columns=categorical_columns, drop_first=True)
    X_test_encoded = pd.get_dummies(X_test, columns=categorical_columns, drop_first=True)
    
    # Display information about the encoded data
    X_train_encoded.info()
    X_test_encoded.info()

    # Ensure both X_train_encoded and X_test_encoded have the same columns
    common_columns = list(set(X_train_encoded.columns) & set(X_test_encoded.columns))
    X_train_final = X_train_encoded[common_columns]
    X_test_final = X_test_encoded[common_columns]
    

    
    



In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Create a Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)

# Fit the classifier to the training data
clf.fit(X_train_final, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test_final)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Get a classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)



In [ ]:
# Get feature importances from the trained Decision Tree Classifier
feature_importances = clf.feature_importances_

# Create a DataFrame to associate feature names with their importances
feature_importance_df = pd.DataFrame({'Feature': X_train_final.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the top N most important features
top_n = 10  # Set the number of top features you want to display
top_features = feature_importance_df.head(top_n)
print(top_features)


Based on the code above, the top ten features deamed important by the Decision Tree Classifier are outputted. It shows that among all the features, the "Estimated graduate enrollment, total" is the most significant feature with a importance value of 0.127.  

Estimated graduate enrollment, total: This feature might have high importance because it could be strongly correlated or indicative of certain types of institutions that fall into specific Carnegie classifications. Institutions with higher graduate enrollment might exhibit characteristics associated with certain categories within the classification.  

SAT Critical Reading 25th percentile score: This could indicate the academic profile of students, which might align with the criteria used in the Carnegie classification system to differentiate between different types of institutions.  

Endowment assets per FTE enrollment: Institutions full-time equivalent (FTE) enrollment might belong to a specific category within the Carnegie classification due to financial resources or institutional characteristics associated with these levels of funding.  

In [ ]:
from sklearn.tree import plot_tree


# Fit the Decision Tree Classifier with max_depth=5
clf = DecisionTreeClassifier(max_depth=5, random_state=42)
clf.fit(X_train_final, y_train)

# Plot the decision tree
plt.figure(figsize=(15, 10))
plot_tree(clf, filled=True, feature_names=X_train_final.columns.tolist(), class_names=[str(c) for c in le.classes_])
plt.show()



In [ ]:
from sklearn.model_selection import GridSearchCV

# Define a range of depths to search through
param_grid = {'max_depth': range(1, 20)}  # Adjust the range as needed

# Create a GridSearchCV object
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5)
grid_search.fit(X_train_final, y_train)

# Get the best estimator and its parameters
dt_best = grid_search.best_estimator_
best_depth = dt_best.get_params()['max_depth']
print(f"Best tree depth: {best_depth}")

# Fit the best model on the entire training data
dt_best.fit(X_train_final, y_train)


In [ ]:
from sklearn.tree import plot_tree

# Plot the final decision tree
plt.figure(figsize=(15, 10))
plot_tree(dt_best, filled=True, feature_names=X_train_final.columns.tolist(), class_names=[str(c) for c in le.classes_])
plt.show()


There seems to be no difference in the dt_best model and the original model.